# Automated Model Testing 

This notebook contains a set of automated tests for the Stroke+Rehab model.  These tests are either pass or fail and no interpretation is needed. A summary of test results is provided at the end of the notebook.

## Imports

In [1]:
import numpy as np
import statistics
from sim_tools.distributions import Lognormal
import pytest
import ipytest
ipytest.autoconfig()

## Model Code Imports

In [2]:
from s2_stroke_rehab_model import *

### Results processing

A set of tests to check that the results of a simulated run are processed correctly for the user.

We test the processing of:

* occupancy frequencies
* probability of delay


In [3]:
@pytest.mark.parametrize('values, rel_expected, cum_expected', [
                          ([1, 1, 1, 1, 2, 2, 2, 3, 3, 4], 
                           [0.4, 0.3, 0.2, 0.1], [0.4, 0.7, 0.9, 1.0])
])
def test_result_processing_1(values, rel_expected, cum_expected):
    '''
    Test the `calculate_occupancy_frequencies` function works
    as expected.

    Expected result: relative frequencies and cumulative freqs
    are the same as expected values.

    Params:
    ------
    values: list
        list of values to test

    rel_expected: list
        list of floats - expected relative freqs

    cum_expected: list
        list of floats - expected cumulative freqs

    Returns:
    -------
    bool: does the model pass the test.
    '''
    rel, cum, unique = calculate_occupancy_frequencies(values)
    # use all close to allow for minor floating point differences.
    assert (set(rel) == set(rel_expected)) and np.allclose(np.array(cum_expected), cum)

In [4]:
@pytest.mark.parametrize('relative, cum, p_delay_expected', [
                          ([0.4, 0.3, 0.2, 0.1], 
                           [0.4, 0.7, 0.9, 1.0], [1.0, 0.3/0.7, 0.2/0.9, 0.1/1.0])
])
def test_result_processing_2(relative, cum, p_delay_expected):
    '''
    Test the probability of delay is calculated correctly
    using the `calculate_prob_delay` function.
    
    Params:
    ------
    relative: list
        list of floats - relative freqs

    cum: list
        list of floats - cumulative freqs

    Returns:
    -------
    bool: does the function pass the test.
    '''
    p_delay = calculate_prob_delay(relative, cum)
    # use all close to allow for minor floating point differences.
    assert np.allclose(np.array(p_delay_expected), p_delay)

### Results collection tests

Test that the optional results collection processes for the ASU and REHAB models work correctly. 

In [5]:
def results_collection_test1(audit_interval=1):
    '''
    Test the model collects acute stroke occupancy every day

    Expected result: len(experiment.asu_occupancy) == env.now

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    # Create the experiment
    experiment = Experiment({
        'run_length': 365*5,  # Run for 5 years
        'trace': False,  # Set to True if you want to see detailed logs
        'acute_audit_interval': audit_interval  # Audit interval as specified
    })

    # Create the simulation environment
    env = simpy.Environment()

    rehab_unit = RehabilitationUnit(env, experiment)

    # Create the AcuteStrokeUnit
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)

    # modified iteration 21
    # start the audit_acute_occupancy to record ASU occupancy at intervals
    env.process(experiment.audit_acute_occupancy(env, 1, audit_interval, asu, experiment))

    # Run the model - modified iteration 21
    asu.run()
    env.run(until=experiment.params['results_collection_period'])

    print(f"Number of occupancy audits: {len(experiment.asu_occupancy)}")
    print(f'Simulation time: {env.now}')
    
    # The number of audits should be equal to the simulation time
    # (assuming audit_interval=1 and the first audit happens at time 1)
    assert len(experiment.asu_occupancy) == (env.now - 1)


In [6]:
def test_results_collection_2(audit_interval=1):
    '''
    Test the model collects rehab occupancy every day

    Expected result: len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    # Create the simulation environment
    env = simpy.Environment()

    # create experiment
    experiment = Experiment()
    
    rehab_unit = RehabilitationUnit(env, experiment)

    # Start the audit process - modified iteration 21
    env.process(experiment.audit_rehab_occupancy(env, 1, experiment.params['rehab_audit_interval'], rehab_unit, experiment))
    
    # Run the model for the default run length in the experiment - modified in iteration 21
    env.run(until=experiment.params['results_collection_period'])
    
    print(f'{len(experiment.rehab_occupancy)=}')
    print(f'{env.now=}')
    assert len(experiment.rehab_occupancy) == (env.now - 1)

In [7]:
def test_results_collection_system(audit_interval=1):
    '''
    SYSTEM TEST
    
    Test REHAB ward occupancy data collected is in a sensible range
    when it is connected to the ASU model.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 1 to [10-15] with sensible moments.

    Expected result: 
        len(experiment.asu_occupancy) == env.now - 1 AND
        len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Create an experiment with default parameters
    experiment = Experiment()

    # Create models
    rehab_unit = RehabilitationUnit(env, experiment)
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    asu.run()
    rehab_unit.run()

    # Start the audit process - modified iteration 21
    env.process(experiment.audit_rehab_occupancy(env, 1, experiment.params['rehab_audit_interval'], rehab_unit, experiment))
    env.process(experiment.audit_acute_occupancy(env, 1, experiment.params['acute_audit_interval'], asu, experiment))
    
    # Run the simulation until the specified run length in the Experiment parameters - modified in iteration 21
    env.run(until=experiment.params['results_collection_period'])

    # Print info for debug
    print(f"Average rehab occupancy: {sum(experiment.rehab_occupancy) / len(experiment.rehab_occupancy):.2f}")
    print(f"Maximum rehab occupancy: {max(experiment.rehab_occupancy)}")
    print(f"Minimum rehab occupancy: {min(experiment.rehab_occupancy)}")
    print(f'{len(experiment.asu_occupancy)=}')
    print(f'{len(experiment.rehab_occupancy)=}')
    print(f'{env.now=}')

    # Test
    assert len(experiment.asu_occupancy) == (env.now - 1) and \
           len(experiment.rehab_occupancy) == (env.now - 1)

## Mode run tests

Here we test that various modes of running the model work correctly.  These include

* results collection period
* warm-up
* single run mode
* repeatable results using random number sets.

In [8]:
def test_results_collection_system(audit_interval=1):
    '''
    SYSTEM TEST
    
    Test REHAB ward occupancy data collected is in a sensible range
    when it is connected to the ASU model.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 1 to [10-15] with sensible moments.

    Expected result: 
        len(experiment.asu_occupancy) == env.now - 1 AND
        len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
   # Create the simulation environment
    env = simpy.Environment()
    
    # Create an experiment with default parameters
    experiment = Experiment()

    # Create models
    rehab_unit = RehabilitationUnit(env, experiment)
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    asu.run()
    rehab_unit.run()

    # Start the audit process - modified iteration 21
    env.process(experiment.audit_rehab_occupancy(env, 1, experiment.params['rehab_audit_interval'], rehab_unit, experiment))
    env.process(experiment.audit_acute_occupancy(env, 1, experiment.params['acute_audit_interval'], asu, experiment))
    
    # Run the simulation until the specified run length in the Experiment parameters - modified in iteration 21
    env.run(until=experiment.params['results_collection_period'])

    # Print info for debug
    print(f"Average rehab occupancy: {sum(experiment.rehab_occupancy) / len(experiment.rehab_occupancy):.2f}")
    print(f"Maximum rehab occupancy: {max(experiment.rehab_occupancy)}")
    print(f"Minimum rehab occupancy: {min(experiment.rehab_occupancy)}")
    print(f'{len(experiment.asu_occupancy)=}')
    print(f'{len(experiment.rehab_occupancy)=}')
    print(f'{env.now=}')

    # Test
    assert len(experiment.asu_occupancy) == (env.now - 1) and \
           len(experiment.rehab_occupancy) == (env.now - 1)


In [9]:
@pytest.mark.parametrize('warm_up, audit_interval', [
                          (365, 1),
                          (1000, 1)
])
def test_warm_up(warm_up, audit_interval):
    '''
    Test warm-up works correctly for ASU+REHAB ward occupancy

    Expected result: 
        len(experiment.asu_occupancy) == experiment.results_collection_period 
        AND len(experiment.rehab_occupancy) == experiment.results_collection_period 

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    # Create the experiment with specified warm-up period
    experiment = Experiment({
        'warm_up': warm_up,
        'acute_audit_interval': audit_interval,
        'rehab_audit_interval': audit_interval
    })

    # Create the simulation environment
    env = simpy.Environment()
    
    # Create models
    rehab_unit = RehabilitationUnit(env, experiment)
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)
    
    # Start the ASU patient generators
    asu.run()

    # Start the REHAB patient generators
    rehab_unit.run()

    # Start the audit processes
    env.process(experiment.audit_acute_occupancy(env, warm_up, audit_interval, asu, experiment))
    env.process(experiment.audit_rehab_occupancy(env, warm_up, audit_interval, rehab_unit, experiment))
    
    # Run the simulation
    env.run(until=experiment.warm_up + experiment.params['results_collection_period'])

    # Print info for debug
    print(f'{len(experiment.asu_occupancy)=}')
    print(f'{len(experiment.rehab_occupancy)=}')
    print(f'{env.now=}')
    print(f'{experiment.params["results_collection_period"]=}')
    print(f'{experiment.warm_up + experiment.params["results_collection_period"]=}')

    # Test
    assert (len(experiment.asu_occupancy) == experiment.params['results_collection_period'] and 
            len(experiment.rehab_occupancy) == experiment.params['results_collection_period'])


In [10]:
def test_single_run():
    '''
    Test the the single_run function returns a dictionary of 
    results.

    The results dictionary contains the following keys:

    'relative_freq_asu'
    'prob_delay_asu'
    'unique_vals_asu'
    'relative_freq_rehab'
    'prob_delay_rehab'
    'unique_vals_rehab'

    Expected result: 
        len(run_results) == 6 and type(run_results) == dict

    Returns:
    -------
    bool: does the model pass the test.
    '''

    # a default experiment
    default_experiment_params = Experiment()

    # run the model
    run_results = single_run(default_experiment_params)

    print(f"{run_results['relative_freq_asu']=}")
    
    # test
    assert len(run_results) == 6 and type(run_results) == dict

### Random number set test (ASU only)

Test that ASU results are repeated each time the same random number set is used.

In [11]:
@pytest.mark.parametrize('random_number_set, print_output', [
                          (0, False),
                          (1, False),
                          (2, False),
                          (101, False),
                          (42, False),
])
def test_random_number_set_1(random_number_set, print_output):
    '''
    Test the the ASU model produces repeatable results. 

    Compares
    min, max, mean of occupancy.
    
    Expected result: 
        set(run1) == set(run2)

    Returns:
    -------
    bool: does the model pass the test.
    '''

    results = []

    for i in range(2):
        
        # Create the simulation environment
        env = simpy.Environment()
        
        # Initialize the Acute Stroke Unit model
        experiment = Experiment(random_number_set=random_number_set)
    
        # Add RU, but do not run the model
        rehab_unit = RehabilitationUnit(env, experiment)
        
        asu = AcuteStrokeUnit(env, experiment, rehab_unit)
        
        # Start the patient generators
        asu.run()
    
        # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
        env.process(experiment.audit_acute_occupancy(env, 1, 1, asu, experiment))
        
        # Run the simulation until the specified run length in the Experiment parameters
        env.run(until=experiment.params['results_collection_period'])

        if print_output: 
            print(f'Run {i} results:')
            print(f'{min(experiment.asu_occupancy)=}')
            print(f'{max(experiment.asu_occupancy)=}')
            print(f'{statistics.fmean(experiment.asu_occupancy)=}')
            print([round(q, 1) for q in statistics.quantiles(experiment.asu_occupancy, n=10)])

        results.append(set((min(experiment.asu_occupancy), 
                            max(experiment.asu_occupancy),
                            statistics.fmean(experiment.asu_occupancy))))
    
    # test
    assert results[0] == results[1]


In [12]:
@pytest.mark.parametrize('random_number_set, print_output', [
                          (0, False),
                          (1, False),
                          (2, False),
                          (101, False),
                          (42, False),
])
def test_random_number_set_2(random_number_set, print_output):
    '''
    Test the the single_run function returns a dictionary of 
    results.

    The results dictionary contains the following keys:

    'relative_freq_asu'
    'prob_delay_asu'
    'unique_vals_asu'
    'relative_freq_rehab'
    'prob_delay_rehab'
    'unique_vals_rehab'

    Expected result: 
        len(run_results) == 6 and type(run_results) == dict

    Returns:
    -------
    bool: does the model pass the test.
    '''
    
    results = []
    for i in range(2):
    
        # set a random number set for streams
        experiment = Experiment(random_number_set=random_number_set)
    
        # run the model
        run_results = single_run(experiment)

        if print_output: 
            print(f'Run {i} results:')
            print(f'{min(experiment.asu_occupancy)=}')
            print(f'{max(experiment.asu_occupancy)=}')
            print(f'{statistics.fmean(experiment.asu_occupancy)=}')
            print([round(q, 1) for q in statistics.quantiles(experiment.asu_occupancy, n=10)])

        results.append(set((min(experiment.asu_occupancy), 
                            max(experiment.asu_occupancy),
                            statistics.fmean(experiment.asu_occupancy))))
    
    # test
    assert results[0] == results[1]

### Lognormal test

Test that lognomal function correctly calculates the moments of the underlying normal dist.

In [13]:
@pytest.mark.parametrize('mean, std', [
                          (128.79, 267.51),
                          (50.0, 2.0),
                          (10.5, 1.0),
])
def test_lognormal_moments(mean, std):
    '''
    Test that lognomal function correctly calculates 
    the moments of the underlying normal dist.

    Params:
    ------
    mean: float
        mean of the lognormal distribution

    std: float
        st dev of the lognormal distribution

    Returns:
    -------
    bool
    '''
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)

    # Convert lognormal parameters from llm for asu
    normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
    normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))
    print(normal_mean, normal_std)

    # Check llm lognormal function used in rehab model - changed in iteration 23
    normal_mean2 = np.log(mean ** 2 / np.sqrt(std ** 2 + mean ** 2))
    normal_std2 = np.sqrt(np.log(std ** 2 / mean ** 2 + 1))
    print (normal_mean2, normal_std2)


    assert (normal_mean, normal_std) == pytest.approx((expected_moments.mu, expected_moments.sigma)) == (normal_mean2, normal_std2)
    

### Extreme value tests

Extreme value tests are used pragmatically to block of routes/arrivals/activites in the simulation model and check the results.

The most simple way to modify the model for these tests is to set parameters to $M$ a very large number.

We test

* Block all arrivals
* Acute LoS is infinite
* Block all but stroke->rehab patient arrivals in the ASU.
* Block all rehab arrivals apart from stroke.
* Block all external rehab arrivals
* Rehab LoS is infinite (Rehab model only)
* Block all arrivals to the ASU and Rehab models

In [14]:
M = 10_000_000

In [15]:
def ev_test_2(large_number=M):
    '''
    All patient types have their inter-arrival time set to $M$ a very large number.
    
    Expected result: No patients arrive to the model.
    
    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    
    # Define custom parameters
    custom_params = {
        'patient_types': {
            'Stroke': {'interarrival_time': large_number},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
    }
    
    # Create an Experiment instance with custom parameters
    experiment = Experiment(custom_params)
    
    # Run the simulation with the custom experiment
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    model = AcuteStrokeUnit(env, experiment, rehab_unit)
    model.run()
    env.run(until=experiment.params['results_collection_period'])
    
    # Print results to check
    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    
    # Return the number of total arrivals

    assert model.total_arrivals == 0


In [16]:
def ev_test_3(large_number=M):
    '''
    All patient types have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: No patients depart the model 
    The number of arrivals = the occupancy of the model.

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    
    # Define custom parameters
    custom_params = {
        'Stroke': {'Rehab': (large_number, 8.6), 'ESD': (large_number, 4.8), 'Other': (large_number, 8.7)},
        'TIA': (large_number, 5.0),
        'Complex Neurological': (large_number, 5.0),
        'Other': (large_number, 5.2)
    }
    
    # Create an Experiment instance with custom parameters
    experiment = Experiment(custom_params)
    
    # Run the simulation with the custom experiment
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    model = AcuteStrokeUnit(env, experiment, rehab_unit)
    model.run()
    env.run(until=experiment.params['results_collection_period'])
    
    # Print results to check
    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    
    # Return the number of total arrivals

    total_arrivals = model.total_arrivals
    final_occupancy = model.occupancy

    return total_arrivals == final_occupancy

In [38]:
def test_ev_4(large_number=M):
    '''
    All patient types apart from stroke-rehab patients
    have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: Only stroke patients depart the
    model.
    (assessed crudely with patient_count > occupancy),
    and also from log - see manual testing notebook.
    
    Params:
    -------
    large_number: int
        M a very large number 
    '''
    # Define custom parameters
    custom_params = {
        'Stroke': {'Rehab': (7.4, 8.6), 'ESD': (large_number, 4.8), 'Other': (large_number, 8.7)},
        'TIA': (large_number, 5.0),
        'Complex Neurological': (large_number, 5.0),
        'Other': (large_number, 5.2)
    }
    
    # Create an Experiment instance with custom parameters
    experiment = Experiment(custom_params)
    
    # Run the simulation with the custom experiment
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    model = AcuteStrokeUnit(env, experiment, rehab_unit)
    model.run()
    env.run(until=experiment.params['results_collection_period'])
    
    # Print results to check
    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    
    # Return the number of total arrivals

    total_arrivals = model.total_arrivals
    final_occupancy = model.occupancy

    assert total_arrivals > final_occupancy

In [39]:
def test_ev_5(large_number=M):
    '''
    Complex Neuro, Other, have their rehab inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the rehab model 
    is "Stroke". This is verified by the patient counts variables in the model.

    Notes:
    ------
    This test will need to be modified when the hardcoded parameters
    are migrated to the Experiment class. 

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    bool: rehab_unit.stroke_count == rehab_unit.patient_count
    '''
    
    experiment = Experiment({
        'trace': False,  # Set to True if you want to see detailed logs
        'rehab_stroke_iat': 21.8,
        'rehab_neuro_iat': large_number,
        'rehab_other_iat': large_number
    })
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    rehab_unit.run()
    env.run(until=experiment.params['results_collection_period'])

    # Print out stats collected
    print(f"Patient counts:")
    for patient_type, count in rehab_unit.patient_counts.items():
        print(f"  {patient_type}: {count}")

    print(f"Total arrivals: {rehab_unit.total_arrivals}")
    
    # Check if only Stroke patients arrived
    assert rehab_unit.patient_counts['Stroke'] == rehab_unit.total_arrivals

In [40]:
def test_ev_6(large_number=M):
    '''
    All patient types have their REHAB inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: No patients arrive to the model
    This is verified by the patient count variables in the model.

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    bool:rehab_unit.patient_count == 0
    '''
    # Create the simulation environment
    experiment = Experiment({
        'trace': True,  # Set to True if you want to see detailed logs
        'rehab_stroke_iat': large_number,
        'rehab_neuro_iat': large_number,
        'rehab_other_iat': large_number
    })
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    rehab_unit.run()
    env.run(until=experiment.params['results_collection_period'])

    # Print out stats collected
    print(f"Patient counts:")
    for patient_type, count in rehab_unit.patient_counts.items():
        print(f"  {patient_type}: {count}")

    print(f"Total arrivals: {rehab_unit.total_arrivals}")
    
    print(f"Total arrivals: {rehab_unit.total_arrivals}")
    
    assert rehab_unit.total_arrivals == 0

In [41]:
def test_ev_7(large_number=M):
    '''
    All patient types have have their mean length 
    of stay time in REHAB set to $M$ a very large number
    
    Expected result: No patients depart the rehab model 
    The occupancy of the model is equal to the no. patient arrivals
    
    Notes:
    -----
    This test will also need to be modified when TIA treatment is added
    and tested when working in connection with the ASU.

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    # Create the simulation environment
    experiment = Experiment({
        'trace': True,  # Set to True if you want to see detailed logs
        'rehab_stroke_esd_los_mean': large_number,
        'rehab_stroke_other_los_mean': large_number,
        'rehab_complex_neuro_los_mean': large_number,
        'rehab_other_los_mean': large_number
    })
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    rehab_unit.run()
    env.run(until=experiment.params['results_collection_period'])

    # Print out stats collected
    print(f"Patient counts:")
    for patient_type, count in rehab_unit.patient_counts.items():
        print(f"  {patient_type}: {count}")

    print(f"Total arrivals: {rehab_unit.total_arrivals}")

    print(f"Total arrivals: {rehab_unit.total_arrivals}")
    
    assert rehab_unit.total_arrivals == rehab_unit.occupancy

In [42]:
def test_ev_8(large_number=M):
    '''
    All patient types have their AUS and REHAB inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: No patients arrive to the model
    This is verified by the patient count variables in the model.

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    bool:rehab_unit.patient_count == 0 and asu.patient_count == 0
    '''
   # Create the simulation environment
    env = simpy.Environment()
    
    # Set all inter-arrival times to a large number
    experiment_params = {
        'run_length': 365,  # Run for 1 year
        'trace': False,
        'patient_types': {
            'Stroke': {'interarrival_time': large_number},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        },
        'rehab_stroke_iat': large_number,
        'rehab_neuro_iat': large_number,
        'rehab_other_iat': large_number
    }
    
    # Create experiment
    experiment = Experiment(experiment_params)
    
    # Create models
    rehab_unit = RehabilitationUnit(env, experiment)
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    asu.run()
    rehab_unit.run()
    
    # Run the simulation
    env.run(until=experiment.params['run_length'])

    # Print out stats collected
    print(f'ASU total arrivals: {asu.total_arrivals}')
    print(f'Rehab total arrivals: {rehab_unit.total_arrivals}')
    for patient_type, count in asu.patient_types.items():
        print(f'ASU {patient_type} count: {count.count}')
    print(f'Rehab patient counts: {rehab_unit.patient_counts}')
    print(f'ASU occupancy: {asu.occupancy}')
    print(f'Rehab occupancy: {rehab_unit.occupancy}')
    
    # Check if all patient counts are zero
    asu_zero = all(pt.count == 0 for pt in asu.patient_types.values())
    rehab_zero = all(count == 0 for count in rehab_unit.patient_counts.values())
    
    assert asu.total_arrivals == rehab_unit.total_arrivals == 0

## Run all automated tests

In [43]:
ipytest.run("-vv", "--no-header")

======================================= test session starts ========================================
collecting ... collected 25 items

t_3b08eb41c6c348babb5152b25997efb1.py::test_result_processing_1[values0-rel_expected0-cum_expected0] PASSED [  4%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_result_processing_2[relative0-cum0-p_delay_expected0] PASSED [  8%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_results_collection_2 PASSED                      [ 12%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_results_collection_system PASSED                 [ 16%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_warm_up[365-1] PASSED                            [ 20%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_warm_up[1000-1] PASSED                           [ 24%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_single_run PASSED                                [ 28%]
t_3b08eb41c6c348babb5152b25997efb1.py::test_random_number_set_1[0-False] PASSED              [ 32%]
t_3b08eb41c6c348babb5152b25997efb1.py

<ExitCode.OK: 0>